### Обязательные библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...);
- [Numpy](http://www.numpy.org) — библиотека для векторных вычислений;
Для выполнения дополнительных заданий (по желанию)
- [Pandas](https://pandas.pydata.org) - библиотека для анализа табличных данных;
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с алгоритмами классического машинного обучения;
- [matplotlib](https://matplotlib.org) - библиотека для построения графиков;

-------

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

# Константы
DATA_DIR = 'data'
RANDOM_STATE = 42

### Модель, генерирующая вектора слов

In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
from utils import download_model

download_model(data_dir=DATA_DIR)

Скачиваем архив из http://vectors.nlpl.eu/repository/20/213.zip...
Архив успешно скачан
Распаковываем архив...
Архив успешно распакован


In [6]:
import gensim

wv_embeddings = gensim.models.fasttext.FastTextKeyedVectors.load(
    DATA_DIR + "/model/model.model"
)

а) из векторного пространства модели

Берётся целевое слово и для него извлекается вектор затем выбирается большое число ближайших кандидатов (topn=500)

Кандидаты фильтруются по порогу косинусной близости sim_threshold=0.48, исключаются словоформы целевого слова — для этого сравнивается одинаковый префикс, также с помощью регулярного выражения убираются токены со знаками препинания и цифрами внутри

Результат функции - список пар (слово, косинусная близость) для слов из всего словаря модели, которые наиболее близки к целевому слову

In [21]:
import re
from typing import List, Tuple

def get_similar_from_model(
    target: str,
    wv_embeddings,
    topn: int = 500,
    sim_threshold: float = 0.4,
    prefix_len: int = 3,
    regex_word: str = r"^[А-Яа-яЁёA-Za-z-]+$"
) -> List[Tuple[str, float]]:
    raw = wv_embeddings.most_similar(target, topn=topn)
    pref = target[:prefix_len].lower()
    result = []
    for w, sim in raw:
        if sim < sim_threshold:
            continue
        if w.lower().startswith(pref):# отсев словоформ
            continue
        if not re.match(regex_word, w):# отсев странных токенов
            continue
        result.append((w, sim))

    return result

In [22]:
target_word = "штат"
similar_model = get_similar_from_model(target_word, wv_embeddings,topn=500, sim_threshold=0.48)
similar_model[:20]

[('калифорния', 0.7500761151313782),
 ('пенсильвания', 0.7051037549972534),
 ('америка', 0.690152645111084),
 ('орегон', 0.6841040849685669),
 ('техас', 0.6772359609603882),
 ('флорида', 0.66839998960495),
 ('канада', 0.6636497974395752),
 ('сша', 0.6608284711837769),
 ('иллинойс', 0.660652756690979),
 ('миннесота', 0.6557710766792297),
 ('айова', 0.6554882526397705),
 ('аризона', 0.6543841361999512),
 ('алабама', 0.6538816094398499),
 ('теннесси', 0.6483874917030334),
 ('нь', 0.644929826259613),
 ('мичиган', 0.6390215754508972),
 ('висконсин', 0.6380236148834229),
 ('каролина', 0.6333165764808655),
 ('невада', 0.6325955986976624),
 ('коннектикут', 0.6264144778251648)]

б)из списка слов, содержащихся в некотором файле

Получаю вектор целевого слова many_meaning_word = wv_embeddings[<слово>]. Читаю файл и разбиваю его на токены. Для каждого слова из файла, которое присутствует в словаре модели, вычисляю косинусную близость между его вектором и вектором слова по sklearn.metrics.pairwise.cosine_similarity

Формирую троеку (слово, вектор, близость) и сортирую по убыванию близости. С помощью цикла ищу пока значение похожести не опускается ниже заданного порога также 0.48. Беру только верхнюю часть списка, получаю список слов и соответствующие им эмбеддинги. В конце нормализую эмбеддинги найденных слов по длине

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similar_from_file(
    target: str,
    wv_embeddings,
    filepath: str = "words_big.txt",
    sim_threshold: float = 0.48,
):
    many_meaning_word = wv_embeddings[target]

    with open(filepath, "r", encoding="utf-8") as reader:
        words = reader.read().split()[1:]

    words_with_similarity = []
    for word in words:
        if word not in wv_embeddings:
            continue
        sim = cosine_similarity(many_meaning_word.reshape(1, -1),wv_embeddings[word].reshape(1, -1))[0, 0]
        words_with_similarity.append((word, wv_embeddings[word], sim))

    words_with_similarity = sorted(words_with_similarity,key=lambda pair: pair[2],reverse=True)

    i = 0
    while i < len(words_with_similarity) and words_with_similarity[i][2] >= sim_threshold:
        i += 1
    split_line = i 

    words_splitted = words_with_similarity[:split_line]
    words_res = []
    embeddings = []
    for (word, emb, _) in words_splitted:
        words_res.append(word)
        embeddings.append(emb)

    embeddings = np.array(embeddings)
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    return words_res, embeddings, words_splitted

Нашла какой-то случайный словарь русских слов на github: https://github.com/danakt/russian-words/blob/master/russian.txt 

Сохраним его как words_big.txt

In [10]:
import requests

url = "https://raw.githubusercontent.com/danakt/russian-words/master/russian.txt"
resp = requests.get(url)
resp.raise_for_status()
with open("words_big.txt", "w", encoding="utf-8") as f:
    f.write(resp.text)

Долго, но зато много слов и работает :)

In [27]:
target_word = "штат"
words_file, emb_file, raw_file = get_similar_from_file(target_word,wv_embeddings,filepath="words_big.txt",sim_threshold=0.48)
print("Найдено слов:", len(words_file))
words_file[:20]

Найдено слов: 740


['штат',
 'калифорния',
 'штаты',
 'америка',
 'Калифорния',
 'нь',
 'каролина',
 'арканзас',
 'Миннесота',
 'юта',
 'Вайоминг',
 'Арканзас',
 'пенсильванцем',
 'Висконсин',
 'миннесотский',
 'Джорджия',
 'Висконсина',
 'чикаго',
 'Филадельфия',
 'Вайоминга']

Сравнение а) и б)

In [31]:
set_model = {w for w, _ in similar_model}
set_file  = set(words_file)

both = set_model & set_file
only_a = set_model - set_file
only_b = set_file - set_model

print("Всего слов в списке a):", len(set_model))
print("Всего слов в списке б):", len(set_file))
print("Пересечение списков:", len(both))


Всего слов в списке a): 294
Всего слов в списке б): 740
Пересечение списков: 79


Примеры общих слов (пересечение):

In [42]:
list(both)[:20]

['иммигрант',
 'бостон',
 'столица',
 'графство',
 'колледж',
 'англия',
 'университет',
 'россия',
 'кантон',
 'нь',
 'стажер',
 'городок',
 'британия',
 'онтарийский',
 'калифорния',
 'бирмингемский',
 'мельбурнский',
 'район',
 'иммигрировать',
 'конгресс']

Примеры слов только из a):

In [39]:
list(only_a)[:20]

['рочестер',
 'санта-фе',
 'высококвалифицировать',
 'анкоридж',
 'филадельфия',
 'санта-крус',
 'абердина',
 'керала',
 'техас',
 'питсбург',
 'огайо',
 'бруклин',
 'пуэрто-рико',
 'принстон',
 'ньюпорт',
 'колорадо',
 'канзася',
 'дублин',
 'оклахом',
 'коннектикута']

Примеры слов только из б):

In [40]:
list(only_b)[:20]

['Кингстон',
 'Висконсин',
 'канберрское',
 'пенсильванские',
 'алабамского',
 'атлантам',
 'калифорниевом',
 'Луизиана',
 'западно',
 'найробийский',
 'миннесотских',
 'Канада',
 'висконсинском',
 'мельбурнском',
 'ассамский',
 'теннесийских',
 'миссисипская',
 'иммиграционном',
 'конгрессмены',
 'миссисипской']